# 教師あり学習と教師なし学習

機械学習の入口で最初に分かれるのが「正解ラベルがあるかどうか」です。
教師あり学習は、入力と正解の対応を学んで予測する枠組みです。教師なし学習は、正解がないデータから構造やまとまりを見つけます。

このノートでは、同じような数値データを使って両者を比較し、
どんな問いにどちらを使うべきかを、コードと結果の対応で確認します。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification, make_blobs, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, silhouette_score

sns.set_theme(style="whitegrid", context="notebook")


## 教師あり学習: ラベルありデータで予測する

まずは二値分類データを作り、`0` と `1` を予測する問題を作ります。
ここでは「ラベルがある」ことが重要で、モデルはこのラベルを目標に学習します。

In [ ]:
X, y = make_classification(
    n_samples=600,
    n_features=6,
    n_informative=4,
    n_redundant=1,
    class_sep=1.2,
    random_state=42,
)

feature_names = [f"x{i}" for i in range(X.shape[1])]
df_cls = pd.DataFrame(X, columns=feature_names)
df_cls["label"] = y

df_cls.head()


学習時に訓練データと評価データを分けるのは、汎化性能を確認するためです。
評価データを分けずに性能を見ると、実運用での精度を過大評価しやすくなります。

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_cls[feature_names],
    df_cls["label"],
    test_size=0.25,
    random_state=42,
    stratify=df_cls["label"],
)

clf = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=1000, random_state=42)),
])
clf.fit(X_train, y_train)

pred = clf.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f"test accuracy: {acc:.3f}")


分類では、正解率だけでなく混同行列を確認すると、
どのクラスで間違っているかを具体的に把握できます。

In [ ]:
cm = confusion_matrix(y_test, pred)
fig, ax = plt.subplots(figsize=(4.5, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax)
ax.set_title("Confusion Matrix")
ax.set_xlabel("Predicted")
ax.set_ylabel("Actual")
plt.show()

print(classification_report(y_test, pred, digits=3))


標準化の有無で精度がどう変わるかを見ると、
前処理がモデル性能に与える影響を体感できます。

In [ ]:
clf_no_scale = LogisticRegression(max_iter=1000, random_state=42)
clf_no_scale.fit(X_train, y_train)
pred_no_scale = clf_no_scale.predict(X_test)

acc_no_scale = accuracy_score(y_test, pred_no_scale)
print(f"without scaling: {acc_no_scale:.3f}")
print(f"with scaling   : {acc:.3f}")


## 教師なし学習: ラベルなしデータで構造を見つける

次はクラスタリングです。ここでは学習時に正解ラベルを使いません。
「似た点を同じグループにまとめる」こと自体が目的になります。

In [ ]:
X_blob, _ = make_blobs(
    n_samples=450,
    centers=3,
    cluster_std=1.2,
    n_features=2,
    random_state=42,
)

fig, ax = plt.subplots(figsize=(5, 4))
ax.scatter(X_blob[:, 0], X_blob[:, 1], s=18, alpha=0.8)
ax.set_title("Unlabeled Data for Clustering")
ax.set_xlabel("feature 1")
ax.set_ylabel("feature 2")
plt.show()


クラスタ数 `k` は先に与える必要があります。
`inertia` と `silhouette` を併せて見ると、極端な `k` を避けやすくなります。

- inertia: 各点と所属クラスタ中心の距離二乗和（小さいほどまとまりが良い）
- silhouette: クラスタ内の近さとクラスタ間の離れ具合のバランス（-1〜1で高いほど良い）

inertia は `k` を増やすと下がりやすいので、silhouette とセットで判断するのが実践的です。

In [ ]:
rows = []
for k in range(2, 8):
    km = KMeans(n_clusters=k, n_init=20, random_state=42)
    labels = km.fit_predict(X_blob)
    rows.append({
        "k": k,
        "inertia": km.inertia_,
        "silhouette": silhouette_score(X_blob, labels)
    })

k_report = pd.DataFrame(rows)
k_report


ここでは `k=3` を選んで可視化します。
教師ありと違い、評価は「正解と一致したか」ではなく、点群のまとまりの良さで判断します。

In [ ]:
kmeans = KMeans(n_clusters=3, n_init=20, random_state=42)
cluster_id = kmeans.fit_predict(X_blob)
centers = kmeans.cluster_centers_

fig, ax = plt.subplots(figsize=(5.5, 4.2))
scatter = ax.scatter(X_blob[:, 0], X_blob[:, 1], c=cluster_id, cmap="tab10", s=22, alpha=0.8)
ax.scatter(centers[:, 0], centers[:, 1], c="black", marker="X", s=140, label="centers")
ax.legend(loc="best")
ax.set_title("KMeans Clustering (k=3)")
ax.set_xlabel("feature 1")
ax.set_ylabel("feature 2")
plt.show()


教師なし学習の代表例として次元圧縮も確認します。
PCA は「情報の分散が大きい方向」を優先して、特徴量を少数次元へ圧縮します。

PCA は特徴量スケールに影響されるため、先に標準化してから実行するのが基本です。
出力される `explained_variance_ratio_` は、各主成分が全体の分散の何割を説明しているかを表します。

In [ ]:
iris = load_iris(as_frame=True)
X_iris = iris.data
y_iris = iris.target

X_iris_scaled = StandardScaler().fit_transform(X_iris)
pca = PCA(n_components=2, random_state=42)
X_iris_2d = pca.fit_transform(X_iris_scaled)

pca_df = pd.DataFrame(X_iris_2d, columns=["PC1", "PC2"])
pca_df["species"] = y_iris.map({0: "setosa", 1: "versicolor", 2: "virginica"})

fig, ax = plt.subplots(figsize=(6, 4.5))
sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="species", ax=ax)
ax.set_title("PCA Projection of Iris")
plt.show()

print("explained variance ratio:", pca.explained_variance_ratio_)


## まとめ

- 教師あり学習: ラベルを使って予測器を作る（分類・回帰）
- 教師なし学習: ラベルなしデータから構造を見つける（クラスタリング・次元圧縮）

実務では、教師なしでデータの構造を把握してから、教師ありで予測モデルを構築する流れもよく使われます。